In [43]:
import yaml
import os
from pathlib import Path
import pandas as pd
import struct
import random
import subprocess
import numpy as np
import matplotlib.pyplot as plt
import sys
import math
import re

# also: cd hammer && pip install -e . 

In [44]:
# experiment setup
PDK = 'sky130'
CLOCK_PERIOD = 10 if PDK == 'sky130' else 2 # ns

# generate custom make str for each test
make_extra = f"pdk={PDK}"
if PDK == 'intech22': make_extra += f" PDK_CONF=experiments/intech22.yml"

# useful paths
energy_char_dpath = Path(os.getcwd()).parent
tests_dpath = energy_char_dpath/f'experiments/tests-{PDK}'
tests_dpath

PosixPath('/bwrcq/scratch/shreyas_thumathy/feb25/hammer/e2e/hammer-energy-char/experiments/tests-sky130')

In [45]:
# get pdk clock periods, paths
PDKs = ["sky130"]
test_paths = {}
for pdk in PDKs:
    test_paths[pdk] = energy_char_dpath/f'experiments/tests-{pdk}'

operations = ['regfile']
modules = {op:op for op in operations}
module_to_inst = {'regfile':'regfile'}
clock_name = {'regfile':'clk'} ## name of clock for respective module name
num_inputs, widths, depth = 1, [], []
for i in [8, 32, 64, 128]:
    for j in [8, 32, 64, 128]:
        widths.append(i)
        depth.append(j)
clock_periods = [CLOCK_PERIOD] # ns, clock periods to apply to all designs; eventually try sweeping this


design_minclock_dict = {}
# optional:
# design_minclock_dict={'add4': 4, 'add8': 4, 'add16': 4, 'add32': 4, 'add64': 4, 'mul4': 5, 'mul8': 5, 'mul16': 6, 'mul32': 6, 'mul64': 5}

'''
tests dict
  name format: <design>-<test_name>
      inputs: list, where each item is a tuple of items per line in input.txt
      defines: for verilog
'''

'\ntests dict\n  name format: <design>-<test_name>\n      inputs: list, where each item is a tuple of items per line in input.txt\n      defines: for verilog\n'

In [46]:
def writeYaml(td):
    defines_str = '\n'.join( [ f"  - {key}={val}" for key,val in td['defines'].items() ] )
    clock_period = td["clock_period"]
    delays = [f"""{{name: {i}, clock: {td['clock']}, delay: "1", direction: input}}""" for i in td['input_ports']]
    delays += [f"""{{name: {i}, clock: {td['clock']}, delay: "1", direction: output}}""" for i in td['output_ports']]
    delays = ',\n  '.join(delays)
    cfg = f"""\
vlsi.core.build_system: make
vlsi.inputs.power_spec_type: cpf
vlsi.inputs.power_spec_mode: auto

design.defines: &DEFINES
  - CLOCK_PERIOD={clock_period}
{defines_str}

vlsi.inputs.clocks: [{{name: {td['clock']}, period: "{clock_period}ns", uncertainty: "100ps"}}]

vlsi.inputs.delays: [
  {delays}
]

synthesis.inputs:
  top_module: {td['top_module']}
  input_files: {td['vsrcs']}
  defines: *DEFINES

sim.inputs:
  top_module: {td['top_module']}
  tb_name: {td['tb_name']}
  tb_dut: {td['tb_dut']}
  options: ["-timescale=1ns/10ps", "-sverilog"]
  options_meta: append
  defines: *DEFINES
  defines_meta: append
  level: rtl
  input_files: {td['vsrcs'] + td['vsrcs_tb']}

vlsi.core.power_tool: hammer.power.joules
power.inputs:
  level: rtl
  top_module: {td['top_module']}
  tb_name: {td['tb_name']}
  tb_dut: {td['tb_dut']}
  defines: *DEFINES
  input_files: {td['vsrcs']}
  report_configs:
    - waveform_path: {td['root']}/output.fsdb
      report_stem: {td['root']}/power
      toggle_signal: {td['clock']}
      num_toggles: 1
      levels: all
      output_formats:
      - report
      - plot_profile
      - ppa
"""
    with (td['root']/'config.yml').open('w') as f:
        f.write(cfg)

In [47]:
def formattedOp(op):
    if (op == 'regfile'):
        return 'regfile'


# Reset regs to 0 or rand
def get_initial_vals(n, width, rand=True):
    vals = []
    for i in range(n):
        rand_val = random.randint(0, 1 << width - 1)
        vals.append([0, 0, 1, i, rand_val if rand else 0])
    return vals

def writeTestSpecial(n, width, rand=True):
    init_vals = get_initial_vals(n, width, False) # always reset to regs to 0
    ret = []
    for i in range(n):
        rand_val = 1 << (random.randint(0, width - 1))
        # r_en, r_addr, w_en, w_addr, w_data 
        ret.append([0, 0, 1, i, rand_val]) # turn 1 bit on
    return init_vals, ret # reset.txt, input.txt vals

def writeTest(n, width, rand=True):
    init_vals = get_initial_vals(n, width, False) # always reset to regs to 0
    ret = []
    for i in range(n):
        rand_val = random.randint(0, 1 << width - 1)
        # r_en, r_addr, w_en, w_addr, w_data 
        ret.append([0, 0, 1, i, rand_val if rand else 1 << width - 1]) # write all 1s or rand val
    return init_vals, ret # reset.txt, input.txt vals

def readTest(n, width, rand=True):
    init_vals = get_initial_vals(n, width, rand) # reset to rand or 0
    ret = []
    for i in range(n):
        # r_en, r_addr, w_en, w_addr, w_data 
        ret.append([1, i, 0, 0, 0])
    return init_vals, ret

def resetTest(n, width, rand=True):
    init_vals = get_initial_vals(n, width, rand)
    ret = []
    for i in range(n):
        # r_en, r_addr, w_en, w_addr, w_data 
        ret.append([0, 0, 1, i, 0])
    return init_vals, ret


def createTest(operation, n, width, clock_period, test_type, rand=False):
    tt_to_name = {resetTest: 'reset', readTest: 'read', writeTest: 'write', writeTestSpecial: 'write'}
    design_name = f"{formattedOp(operation)}{n}x{width}"
    design = f"{design_name}-{clock_period}ns"
    test_name = f'{design}-{tt_to_name[test_type]}'
    if (rand):
        test_name = f'{test_name}-rand'
    if (not rand and test_type == writeTest):
        test_name = f'{test_name}-af-{width}bit'
    if (test_type == writeTestSpecial):
        test_name = f'{test_name}-af-1bit'


    reset_vals, inputs = test_type(n, width, rand)
    print("Operation, #registers, width:", operation, n, width)
    print("For test type:", test_type)
    print("Reset values:", reset_vals)
    print("Input values:", inputs)
    ## set inputs
    new_test = {'inputs': inputs, 'resets': reset_vals, 'defines': dict(WIDTH=width)}

    new_test['depth'] = n
    new_test['width'] = width

    ## verilog info
    module = modules[operation]
    new_test['design_name'] = design_name
    new_test['test_name'] = test_name
    new_test['design'] = design
    new_test['inst'] = f'/{module_to_inst[module]}'
    new_test['clock'] = clock_name[module]
    new_test['vsrcs'] = [f'src/{module}.v']
    new_test['vsrcs_tb'] = [f'src/{module}_tb.v']
    new_test['top_module'] = f'{module}'
    new_test['tb_name'] = f'{module}_tb'
    new_test['tb_dut'] = f'{module}_dut'
    new_test['input_ports'] = ['R_addr', 'R_en', 'R_data', 'W_addr', 'W_en', 'W_data']
    new_test['output_ports'] = ['R_data']
    new_test['read'] = (test_type == readTest)
    new_test['clock_period'] = clock_period

    ## root dir
    root = tests_dpath/test_name
    root.mkdir(exist_ok=True,parents=True)
    new_test['defines']['TESTROOT'] = root
    new_test['defines']['WIDTH'] = width
    new_test['defines']['N'] = n
    new_test['defines']['read'] = new_test['read']
    new_test['root'] = root

    ## design dir
    new_test['obj_dpath'] = energy_char_dpath/f"build-{PDK}-cm/{design}"

    cfg = str(new_test['root']/'config.yml')
    new_test['make'] = f"design={new_test['design']} {make_extra} DESIGN_CONF={cfg}"

    return test_name, new_test

# convert data operands to binary format to dump to input.txt (gets more complicated for floats)
def val2binary(val) -> str:
    if type(val) == str: return val
    elif type(val) == int: return '{0:b}'.format(val)
    else: assert(False), f"Invalid dtype, {type(val)}"

## write inputs to test file
def writeInput(test_info):
    with (test_info['root']/'input.txt').open('w') as f:
        for operands in test_info['inputs']:
            f.write(" ".join([val2binary(operand) for operand in operands]) + '\n')

    with (test_info['root']/'reset.txt').open('w') as f:
        for operands in test_info['resets']:
            f.write(" ".join([val2binary(operand) for operand in operands]) + '\n')
    

In [48]:
tests_dict = {}
for i in range(len(operations)):
    operation = operations[i]
    width = widths[i]
    for n in depth:
        design = operation
        design_clock_periods = clock_periods.copy()
        if design in design_minclock_dict:
            design_clock_periods.append(design_minclock_dict[design])
        for clock_period in design_clock_periods:
                    for tt in [resetTest, readTest, writeTest, writeTestSpecial]:
                        for rand_val in [True, False]:
                            print("Working on", operation, n, width, clock_period)
                            test_name, test_info = createTest(operation, n, width, clock_period, tt, rand_val)
                            writeInput(test_info)
                            writeYaml(test_info)
                            tests_dict[test_name] = test_info
                            print(test_info)

Working on regfile 8 8 10
Operation, #registers, width: regfile 8 8
For test type: <function resetTest at 0x7f69abb48040>
Reset values: [[0, 0, 1, 0, 20], [0, 0, 1, 1, 84], [0, 0, 1, 2, 39], [0, 0, 1, 3, 84], [0, 0, 1, 4, 77], [0, 0, 1, 5, 3], [0, 0, 1, 6, 66], [0, 0, 1, 7, 79]]
Input values: [[0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [0, 0, 1, 2, 0], [0, 0, 1, 3, 0], [0, 0, 1, 4, 0], [0, 0, 1, 5, 0], [0, 0, 1, 6, 0], [0, 0, 1, 7, 0]]
{'inputs': [[0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [0, 0, 1, 2, 0], [0, 0, 1, 3, 0], [0, 0, 1, 4, 0], [0, 0, 1, 5, 0], [0, 0, 1, 6, 0], [0, 0, 1, 7, 0]], 'resets': [[0, 0, 1, 0, 20], [0, 0, 1, 1, 84], [0, 0, 1, 2, 39], [0, 0, 1, 3, 84], [0, 0, 1, 4, 77], [0, 0, 1, 5, 3], [0, 0, 1, 6, 66], [0, 0, 1, 7, 79]], 'defines': {'WIDTH': 8, 'TESTROOT': PosixPath('/bwrcq/scratch/shreyas_thumathy/feb25/hammer/e2e/hammer-energy-char/experiments/tests-sky130/regfile8x8-10ns-reset-rand'), 'N': 8, 'read': False}, 'depth': 8, 'width': 8, 'design_name': 'regfile8x8', 'test_name': 'regf

In [49]:
# Setup
def runMakeCmd(make_target,td,fp,overwrite=True,verbose=False):
    if overwrite or not fp.exists():
        cmd = f"make {make_target} {td['make']}"
        print(f'Executing commmand: {cmd}')
        subprocess.run(cmd, cwd=energy_char_dpath,
                        shell=True, check=True, capture_output=(not verbose))

# subprocess.run starts in an empty environment, need to ensure it can find hammer-vlsi
python_exec_fpath = Path(sys.executable)
env_dpath = str(python_exec_fpath.parent)
if not os.environ['PATH'].startswith(env_dpath): os.environ['PATH'] = env_dpath + ':' + os.environ['PATH']

def runBuild(td,overwrite=True,verbose=False):
    runMakeCmd("build -B",td,
                td['obj_dpath'],
                overwrite,verbose)

def runSim(td,overwrite=True,verbose=False):
    runMakeCmd("redo-sim-rtl",td,
                td['root']/'output.fsdb',
                overwrite,verbose)

def runPowerSyn(td,overwrite=True,verbose=False):
    runMakeCmd("power-rtl",td,
                td['obj_dpath']/'power-rtl-rundir/pre_report_power',
                overwrite,verbose)

def runPowerReport(td,overwrite=True,verbose=False):
    runMakeCmd("redo-power-rtl args='--only_step report_power'",td,
                td['root']/'power.power.rpt',
                overwrite,verbose)


In [50]:
# build
overwrite = True
build_dpaths = {td['obj_dpath']: t for t,td in tests_dict.items()} # run build once per build dir (not once per test)
for bd,t in build_dpaths.items():
    runBuild(tests_dict[t],overwrite)

Executing commmand: make build -B design=regfile8x8-10ns pdk=sky130 DESIGN_CONF=/bwrcq/scratch/shreyas_thumathy/feb25/hammer/e2e/hammer-energy-char/experiments/tests-sky130/regfile8x8-10ns-write-af-1bit/config.yml
Executing commmand: make build -B design=regfile32x8-10ns pdk=sky130 DESIGN_CONF=/bwrcq/scratch/shreyas_thumathy/feb25/hammer/e2e/hammer-energy-char/experiments/tests-sky130/regfile32x8-10ns-write-af-1bit/config.yml
Executing commmand: make build -B design=regfile64x8-10ns pdk=sky130 DESIGN_CONF=/bwrcq/scratch/shreyas_thumathy/feb25/hammer/e2e/hammer-energy-char/experiments/tests-sky130/regfile64x8-10ns-write-af-1bit/config.yml
Executing commmand: make build -B design=regfile128x8-10ns pdk=sky130 DESIGN_CONF=/bwrcq/scratch/shreyas_thumathy/feb25/hammer/e2e/hammer-energy-char/experiments/tests-sky130/regfile128x8-10ns-write-af-1bit/config.yml


In [51]:
# power-rtl: synthesize design in power tool + save checkpoint
overwrite = True
for bd,t in build_dpaths.items():
    runSim(tests_dict[t],overwrite)
    runPowerSyn(tests_dict[t],overwrite)
    runPowerReport(tests_dict[t],overwrite)

Executing commmand: make redo-sim-rtl design=regfile8x8-10ns pdk=sky130 DESIGN_CONF=/bwrcq/scratch/shreyas_thumathy/feb25/hammer/e2e/hammer-energy-char/experiments/tests-sky130/regfile8x8-10ns-write-af-1bit/config.yml
Executing commmand: make power-rtl design=regfile8x8-10ns pdk=sky130 DESIGN_CONF=/bwrcq/scratch/shreyas_thumathy/feb25/hammer/e2e/hammer-energy-char/experiments/tests-sky130/regfile8x8-10ns-write-af-1bit/config.yml
Executing commmand: make redo-power-rtl args='--only_step report_power' design=regfile8x8-10ns pdk=sky130 DESIGN_CONF=/bwrcq/scratch/shreyas_thumathy/feb25/hammer/e2e/hammer-energy-char/experiments/tests-sky130/regfile8x8-10ns-write-af-1bit/config.yml
Executing commmand: make redo-sim-rtl design=regfile32x8-10ns pdk=sky130 DESIGN_CONF=/bwrcq/scratch/shreyas_thumathy/feb25/hammer/e2e/hammer-energy-char/experiments/tests-sky130/regfile32x8-10ns-write-af-1bit/config.yml
Executing commmand: make power-rtl design=regfile32x8-10ns pdk=sky130 DESIGN_CONF=/bwrcq/scratc

In [52]:
# # TODO: check ppa.rpt & adjust clock period, print updated values
# #       in a dict that user copies to the top? it's a bit messy but we prolly don't want to auto-loop

# def parseSlack(fname):
#     slacks = []
#     with open(fname, 'r') as f:
#         lines = f.readlines()
#         for line in lines:
#             line = line.split()
#             if (line[0] == 'register'):
#                 slacks.append(float(line[6]))
#     return min(slacks)

# overwrite = True
# new_clocks = {}
# for bd,t in build_dpaths.items():
#     cur_test = tests_dict[t]
#     clock_period = cur_test['clock_period']
#     slack = parseSlack(cur_test['root']/'power.ppa.rpt')
#     if 0 <= slack and slack < 500: slack = 0 # don't change clock period
#     new_clock_period = math.floor((clock_period * 1000 - slack) / 1000)
#     new_clocks[t.split('-')[0]] = new_clock_period

# print(f"design_minclock_dict={new_clocks}")
# # copy output to top if desired

In [53]:
# sim-rtl
for t,td in tests_dict.items():
    runSim(td, overwrite=True, verbose=True)
    runPowerReport(td, overwrite=True, verbose=True)

Executing commmand: make redo-sim-rtl design=regfile8x8-10ns pdk=sky130 DESIGN_CONF=/bwrcq/scratch/shreyas_thumathy/feb25/hammer/e2e/hammer-energy-char/experiments/tests-sky130/regfile8x8-10ns-reset-rand/config.yml
/bwrcq/scratch/shreyas_thumathy/miniforge3/bin/hammer-vlsi -e /bwrcq/scratch/shreyas_thumathy/feb25/hammer/e2e/configs-env/bwrc-env.yml -p /bwrcq/scratch/shreyas_thumathy/feb25/hammer/e2e/configs-pdk/sky130.yml -p /bwrcq/scratch/shreyas_thumathy/feb25/hammer/e2e/configs-tool/cm.yml -p /bwrcq/scratch/shreyas_thumathy/feb25/hammer/e2e/hammer-energy-char/experiments/tests-sky130/regfile8x8-10ns-write-af-1bit/config.yml -p /bwrcq/scratch/shreyas_thumathy/feb25/hammer/e2e/hammer-energy-char/build-sky130-cm/regfile8x8-10ns/sram_generator-output.json  -p /bwrcq/scratch/shreyas_thumathy/feb25/hammer/e2e/hammer-energy-char/../configs-pdk/sky130.yml  -p /bwrcq/scratch/shreyas_thumathy/feb25/hammer/e2e/hammer-energy-char/../configs-tool/cm.yml  -p /bwrcq/scratch/shreyas_thumathy/feb25/

[tech] Fixing broken net references with select specify blocks.
[tech] Modifying Verilog netlist: /tools/C/openeda/conda-sky130/share/pdk/sky130A/libs.ref/sky130_fd_sc_hd/verilog/primitives.v -> /bwrcq/scratch/shreyas_thumathy/feb25/hammer/e2e/hammer-energy-char/build-sky130-cm/regfile8x8-10ns/tech-sky130-cache/primitives.v
[tech] Modifying Technology LEF: /tools/C/openeda/conda-sky130/share/pdk/sky130A/libs.ref/sky130_fd_sc_hd/techlef/sky130_fd_sc_hd__nom.tlef -> /bwrcq/scratch/shreyas_thumathy/feb25/hammer/e2e/hammer-energy-char/build-sky130-cm/regfile8x8-10ns/tech-sky130-cache/sky130_fd_sc_hd__nom.tlef
[tech] Modifying IO LEF: /tools/C/openeda/conda-sky130/share/pdk/sky130A/libs.ref/sky130_fd_io/lef/sky130_ef_io.lef -> /bwrcq/scratch/shreyas_thumathy/feb25/hammer/e2e/hammer-energy-char/build-sky130-cm/regfile8x8-10ns/tech-sky130-cache/sky130_ef_io.lef
[tech] Fixing broken sky130_ef_io__analog_esd_pad LEF definition.
[tech] Loaded Sky130 Tech
[<global>] Starting simulation with tool 

In [54]:
def parse_hier_power_rpt(td) -> list:
    fpath = td['root']/'power.hier.power.rpt'
    with fpath.open('r') as f: lines = f.readlines()
    for l in lines:
        words = l.split()
        if l.startswith('Power Unit'):
            assert(words[-1] == 'mW'), f"Wrong power unit in report, {l}"
        if td['inst'] == words[-1]:
            #print([float(p) for p in words[2:6]])
            return [float(p) for p in words[2:6]]
    return []

def parse_power_profile(td) -> float:
    fpath = td['root']/'power.profile.png.data'
    with fpath.open('r') as f: lines = f.readlines()
    header = lines[0]
    assert((f"-ykeylabel {td['inst']}:total:total" in header) or (f"-ykeylabel {td['inst'][1:]}:total:total" in header)) # make sure we got the correct power trace
    match = re.search(r'simulation time \((\w+)\)', header)
    unit = match.group(1) if match else None
    if unit == 'ns': scaling = 1
    elif unit == 'ps': scaling = 1e-3
    elif unit == 'fs': scaling = 1e-6
    else: raise ValueError(f"Unit {unit} not supported in file {fpath}")
    time_power = [l.split() for l in lines]
    time_power = [tp for tp in time_power if len(tp) == 2]
    times = [float(t)*scaling for t,p in time_power]
    powers = [float(p) for t,p in time_power][1:-1] # skip first/last values bc they're misleading
    avgpow = sum(powers)/len(powers)
    return times[-1]-times[0], avgpow # end-start


power = []

for t,td in tests_dict.items():
    power.append(parse_hier_power_rpt(td))

power = pd.DataFrame(power,   #  mW
                     columns=['Leakage','Internal','Switching','Total'],
                     index=tests_dict.keys()) # type: ignore


time_ns = [parse_power_profile(td)[0] for td in tests_dict.values()]
energy = power.mul(time_ns,axis=0) / num_inputs # mW * ns = pJ
energy.columns = [c+' Energy (pJ)' for c in energy.columns]

props = pd.DataFrame(index=tests_dict.keys())
props['design'] = [td['design_name'] for td in tests_dict.values()]
props['test'] = [td['test_name'].replace(td['design']+'-','') for td in tests_dict.values()]
props['width'] = [td['width'] for td in tests_dict.values()]
props['depth'] = [td['depth'] for td in tests_dict.values()]
props['time_ns'] = time_ns

database = pd.concat([props,energy,power],axis=1)


#print(results_for_plots)
database.to_hdf(PDK+'.h5',key='df',mode='w')
database


,design,test,width,depth,time_ns,Leakage Energy (pJ),Internal Energy (pJ),Switching Energy (pJ),Total Energy (pJ),Leakage,Internal,Switching,Total
regfile8x8-10ns-reset-rand,regfile8x8,reset-rand,8,8,75.0,0.000064,5.383095,1.604430,6.987585,8.597050e-07,0.071775,0.021392,0.093168
regfile8x8-10ns-reset,regfile8x8,reset,8,8,75.0,0.000064,4.640212,1.448685,6.088957,8.521520e-07,0.061870,0.019316,0.081186
regfile8x8-10ns-read-rand,regfile8x8,read-rand,8,8,75.0,0.000065,3.780458,38.725500,42.506025,8.666760e-07,0.050406,0.516340,0.566747
regfile8x8-10ns-read,regfile8x8,read,8,8,75.0,0.000065,1.895415,1.715475,3.610957,8.601860e-07,0.025272,0.022873,0.048146
regfile8x8-10ns-write-rand,regfile8x8,write-rand,8,8,75.0,0.000065,5.577413,1.992960,7.570425,8.673930e-07,0.074366,0.026573,0.100939
regfile8x8-10ns-write-af-8bit,regfile8x8,write-af-8bit,8,8,75.0,0.000064,4.962008,1.506975,6.469050,8.535420e-07,0.066160,0.020093,0.086254
regfile8x8-10ns-write-rand-af-1bit,regfile8x8,write-rand-af-1bit,8,8,75.0,0.000064,5.154660,1.732995,6.887722,8.583280e-07,0.068729,0.023107,0.091836
regfile8x8-10ns-write-af-1bit,regfile8x8,write-af-1bit,8,8,75.0,0.000064,5.231670,1.823273,7.055010,8.582040e-07,0.069756,0.024310,0.094067
regfile32x8-10ns-reset-rand,regfile32x8,reset-rand,8,32,315.0,0.000961,44.770005,18.688918,63.459900,3.049550e-06,0.142127,0.059330,0.201460
regfile32x8-10ns-reset,regfile32x8,reset,8,32,315.0,0.000950,42.263865,18.120973,60.385815,3.015820e-06,0.134171,0.057527,0.191701


In [55]:
database[database['test'] == 'write-rand']

,design,test,width,depth,time_ns,Leakage Energy (pJ),Internal Energy (pJ),Switching Energy (pJ),Total Energy (pJ),Leakage,Internal,Switching,Total
regfile8x8-10ns-write-rand,regfile8x8,write-rand,8,8,75.0,0.000065,5.577413,1.992960,7.570425,8.673930e-07,0.074366,0.026573,0.100939
regfile32x8-10ns-write-rand,regfile32x8,write-rand,8,32,315.0,0.000987,53.636310,28.985513,82.622925,3.133240e-06,0.170274,0.092018,0.262295
regfile64x8-10ns-write-rand,regfile64x8,write-rand,8,64,635.0,0.003928,180.626385,102.356920,282.987115,6.185180e-06,0.284451,0.161192,0.445649
regfile128x8-10ns-write-rand,regfile128x8,write-rand,8,128,1275.0,0.015275,679.534200,420.549825,1100.099325,1.198010e-05,0.532968,0.329843,0.862823


In [56]:
ykey = 'Switching Energy (pJ)'
# ykey = 'Internal (pJ)'

plt.figure(figsize=(5,4.2))

width_markers = {8: 'v', 16: 'o', 32:'P', 64: '^'}
operation_colors = dict(add='tab:orange',mul='tab:blue',fp_add='tab:orange', fp_mul='tab:blue', bf_mul='tab:blue', bf_add='tab:orange')
operation_linestyles = dict(add='-',mul='--',fp_add='--',fp_mul="-", bf_add=':', bf_mul=':')


designs =  list(dict.fromkeys(database['design']).keys())
print(designs)
for design in designs:
    bf = '_bf' in design
    df = database[(database['design'] == design)].sort_values('output_af')
    hd = df['output_af']
    t = df.index[0]
    width = tests_dict[t]['width']
    op = tests_dict[t]['operation']
    e = df[ykey]
    if ('-rand' in e):
        continue
    plt.plot(hd,e,label=design,
            color=operation_colors[op],
            linewidth=2,
            linestyle=operation_linestyles[op],
            markersize=8,
            marker=width_markers[width] if not bf else 'x')
    plt.xticks(hd)

handles, labels = plt.gca().get_legend_handles_labels()
plt.legend(handles[::-1],labels[::-1],bbox_to_anchor=(1,1),title='Designs',handlelength=4)
plt.xlabel('Output Data Activity Factor')
plt.ylabel(ykey + ' [log scale]')
plt.grid(axis='both')
# plt.ylim(bottom=0)
plt.yscale('log')
;

['regfile8x8', 'regfile32x8', 'regfile64x8', 'regfile128x8']


KeyError: 'output_af'

<Figure size 500x420 with 0 Axes>